In [1]:
import pandas as pd
import numpy as np

## read csv file

In [2]:
df = pd.read_csv('Tweets_cleaned_data.csv')
df.sample(5)

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1
1229582,4,1991400983,Mon Jun 01 06:58:25 PDT 2009,NO_QUERY,sweeetbutterfly,@qutes0911 i know right! work work work work w...,i know right work work work work work but than...,i know right work work work work work but than...,NaN,"['i', 'know', 'right', 'work', 'work', 'work',...","['know', 'right', 'work', 'work', 'work', 'wor...",15
421599,0,2062455424,Sat Jun 06 23:16:52 PDT 2009,NO_QUERY,bright_side,@michaelmillman better than iphone? i can't us...,better than iphone i cant use eitherbecause of...,better than i cant use of my carrier i am resi...,NaN,"['better', 'than', 'i', 'cant', 'use', 'of', '...","['better', 'cant', 'use', 'carrier', 'resistin...",6
1267749,4,1999912360,Mon Jun 01 21:03:03 PDT 2009,NO_QUERY,YourFav_Redhead,"@ginger_swan oh deff, will send more trivia yo...",oh deff will send more trivia your way,oh will send more trivia your way,NaN,"['oh', 'will', 'send', 'more', 'trivia', 'your...","['oh', 'send', 'trivia', 'way']",4
771835,0,2302519819,Tue Jun 23 16:55:32 PDT 2009,NO_QUERY,SaMannequiN,http://twitpic.com/88hzj - my finger is bleedi...,my finger is bleeding profusely,my finger is bleeding profusely,NaN,"['my', 'finger', 'is', 'bleeding', 'profusely']","['finger', 'bleeding', 'profusely']",3
1082107,4,1968664011,Fri May 29 21:55:51 PDT 2009,NO_QUERY,gacconsultants,@randum20 hello randy; appreciate the shout an...,hello randy appreciate the shout and look forw...,hello randy appreciate the shout and look forw...,NaN,"['hello', 'randy', 'appreciate', 'the', 'shout...","['hello', 'randy', 'appreciate', 'shout', 'loo...",8


## Hashtags

### split for fine hashtage

In [3]:
import re
def split_hashtag(hashtag):
    return re.findall(r"#(\w+)", hashtag)

df['hashtags'] = df['text'].apply(split_hashtag)

In [4]:
# make hashtags in each tweet not duplicated
df['hashtags'] = df['hashtags'].apply(lambda x: list(set(x)))

In [5]:
# show where have hashtags
df['hashtags'].apply(lambda x: len(x) > 0).value_counts()

False    1564153
True       35847
Name: hashtags, dtype: int64

In [6]:
# show the number of hashtags in each tweet
df['num_hashtags'] = df['hashtags'].apply(lambda x: len(x))

### look hashtags

In [7]:
# create an array of hashtags
hashtags = np.array([hashtag for hashtag_list in df['hashtags'][pd.notnull(df['hashtags'])] for hashtag in hashtag_list])

In [8]:
len(hashtags)

41225

In [9]:
# count the number of each hashtag
from collections import Counter
hashtag_counts = Counter(hashtags)

# show the top 10 hashtags
hashtag_counts.most_common(10)


[('followfriday', 2620),
 ('fb', 1786),
 ('ff', 973),
 ('squarespace', 924),
 ('1', 516),
 ('iranelection', 516),
 ('musicmonday', 421),
 ('fail', 360),
 ('2', 335),
 ('asot400', 331)]

In [10]:
# if want all hashtags, and sort max -> min
#hashtag_counts.most_common(len(hashtag_counts))

In [11]:
# create a dataframe of hashtags
hashtag_df = pd.DataFrame(hashtag_counts.most_common(len(hashtag_counts)), columns=['hashtag', 'count'])

In [12]:
# save the dataframe to csv file
# hashtag_df.to_csv('hashtag_counts.csv', index=False)

## Sentiment

In [28]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import tensorflow as tf


c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Instantiate Model

In [29]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
## cardiffnlp/twitter-roberta-base-sentiment


In [15]:
df_sentiment = pd.DataFrame()
df_sentiment = df

In [16]:
df_sentiment.head()

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1,num_hashtags
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - awww, t...",awww thats a bummer you shoulda got david carr...,thats a bummer you got carr of third day to do...,[],"['thats', 'a', 'bummer', 'you', 'got', 'carr',...","['thats', 'bummer', 'got', 'carr', 'third', 'd...",6,0
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his facebook by ...,is upset that he cant update his facebook by t...,is upset that he cant update his by it and mig...,[],"['is', 'upset', 'that', 'he', 'cant', 'update'...","['upset', 'cant', 'update', 'might', 'cry', 'r...",10,0
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@kenichan i dived many times for the ball. man...,i dived many times for the ball managed to sav...,i many times for the ball to save the rest go ...,[],"['i', 'many', 'times', 'for', 'the', 'ball', '...","['many', 'times', 'ball', 'save', 'rest', 'go']",6,0
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body itchy and like its on fire,[],"['my', 'whole', 'body', 'itchy', 'and', 'like'...","['whole', 'body', 'itchy', 'like', 'fire']",5,0
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...,no its not at all mad why am i here because i ...,[],"['no', 'its', 'not', 'at', 'all', 'mad', 'why'...","['mad', 'cant', 'see']",3,0


In [17]:
df_sentiment.shape

(1600000, 13)

#### Encode and Calculate Sentiment

In [25]:
def sentiment_score(text):
    if isinstance(text, str):
        # If input is a single string, tokenize it and encode it as a tensor
        tokens = tokenizer.encode(text, return_tensors='pt')
    elif isinstance(text, list):
        # If input is a list of strings, tokenize each string and concatenate the tensors
        tokens = torch.cat([tokenizer.encode(t, return_tensors='pt') for t in text])
    else:
        return -1
        
    result = model(tokens)
    # return int(torch.argmax(result.logits))+1
    # return mean of all the scores
    return int(torch.argmax(result.logits))+1

In [19]:
df_sentiment.head()

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1,num_hashtags
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - awww, t...",awww thats a bummer you shoulda got david carr...,thats a bummer you got carr of third day to do...,[],"['thats', 'a', 'bummer', 'you', 'got', 'carr',...","['thats', 'bummer', 'got', 'carr', 'third', 'd...",6,0
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his facebook by ...,is upset that he cant update his facebook by t...,is upset that he cant update his by it and mig...,[],"['is', 'upset', 'that', 'he', 'cant', 'update'...","['upset', 'cant', 'update', 'might', 'cry', 'r...",10,0
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@kenichan i dived many times for the ball. man...,i dived many times for the ball managed to sav...,i many times for the ball to save the rest go ...,[],"['i', 'many', 'times', 'for', 'the', 'ball', '...","['many', 'times', 'ball', 'save', 'rest', 'go']",6,0
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body itchy and like its on fire,[],"['my', 'whole', 'body', 'itchy', 'and', 'like'...","['whole', 'body', 'itchy', 'like', 'fire']",5,0
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...,no its not at all mad why am i here because i ...,[],"['no', 'its', 'not', 'at', 'all', 'mad', 'why'...","['mad', 'cant', 'see']",3,0


In [20]:
df_sentiment = df_sentiment.dropna(subset=['text3'])
# df_sentiment = df_sentiment.sample(frac=0.2).reset_index(drop=True)

In [21]:
df_sentiment.head()

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1,num_hashtags
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - awww, t...",awww thats a bummer you shoulda got david carr...,thats a bummer you got carr of third day to do...,[],"['thats', 'a', 'bummer', 'you', 'got', 'carr',...","['thats', 'bummer', 'got', 'carr', 'third', 'd...",6,0
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his facebook by ...,is upset that he cant update his facebook by t...,is upset that he cant update his by it and mig...,[],"['is', 'upset', 'that', 'he', 'cant', 'update'...","['upset', 'cant', 'update', 'might', 'cry', 'r...",10,0
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@kenichan i dived many times for the ball. man...,i dived many times for the ball managed to sav...,i many times for the ball to save the rest go ...,[],"['i', 'many', 'times', 'for', 'the', 'ball', '...","['many', 'times', 'ball', 'save', 'rest', 'go']",6,0
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body itchy and like its on fire,[],"['my', 'whole', 'body', 'itchy', 'and', 'like'...","['whole', 'body', 'itchy', 'like', 'fire']",5,0
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...,no its not at all mad why am i here because i ...,[],"['no', 'its', 'not', 'at', 'all', 'mad', 'why'...","['mad', 'cant', 'see']",3,0


In [22]:
df_sentiment.shape

(1583226, 13)

In [23]:
df_sentiment['sentiment_score'] = df_sentiment['text3'].apply(lambda x: sentiment_score(x))

C:\Users\USER\AppData\Local\Temp\ipykernel_2016\3772641994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['sentiment_score'] = df_sentiment['text3'].apply(lambda x: sentiment_score(x))


In [24]:
df_sentiment.shape

(1583226, 14)

In [25]:
df_sentiment.head(10)

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1,num_hashtags,sentiment_score
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - awww, t...",awww thats a bummer you shoulda got david carr...,thats a bummer you got carr of third day to do...,[],"['thats', 'a', 'bummer', 'you', 'got', 'carr',...","['thats', 'bummer', 'got', 'carr', 'third', 'd...",6,0,3
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his facebook by ...,is upset that he cant update his facebook by t...,is upset that he cant update his by it and mig...,[],"['is', 'upset', 'that', 'he', 'cant', 'update'...","['upset', 'cant', 'update', 'might', 'cry', 'r...",10,0,1
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@kenichan i dived many times for the ball. man...,i dived many times for the ball managed to sav...,i many times for the ball to save the rest go ...,[],"['i', 'many', 'times', 'for', 'the', 'ball', '...","['many', 'times', 'ball', 'save', 'rest', 'go']",6,0,2
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body itchy and like its on fire,[],"['my', 'whole', 'body', 'itchy', 'and', 'like'...","['whole', 'body', 'itchy', 'like', 'fire']",5,0,2
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...,no its not at all mad why am i here because i ...,[],"['no', 'its', 'not', 'at', 'all', 'mad', 'why'...","['mad', 'cant', 'see']",3,0,1
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@kwesidei not the whole crew,not the whole crew,not the whole crew,[],"['not', 'the', 'whole', 'crew']","['whole', 'crew']",2,0,2
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,need a hug,need a hug,need a hug,[],"['need', 'a', 'hug']","['need', 'hug']",2,0,5
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@loltrish hey long time no see! yes.. rains a...,hey long time no see yes rains a bit only a bi...,hey long time no see yes a bit only a bit fine...,[],"['hey', 'long', 'time', 'no', 'see', 'yes', 'a...","['hey', 'long', 'time', 'see', 'yes', 'bit', '...",9,0,4
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@tatiana_k nope they didn't have it,nope they didnt have it,nope they didnt have it,[],"['nope', 'they', 'didnt', 'have', 'it']","['nope', 'didnt']",2,0,1
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?,que me muera,me,[],['me'],[],0,0,3


In [26]:
# show the number of each sentiment score
df_sentiment['sentiment_score'].value_counts()

5    562174
1    430789
3    287646
4    159422
2    143195
Name: sentiment_score, dtype: int64

In [27]:
# save the dataframe to csv file
df_sentiment.to_csv('Tweets_sentiment_16M.csv', index=False)

In [28]:
# sort the dataframe by sentiment score
# df_sentiment.sort_values(by=['sentiment_score'], inplace=True, ascending=False)
# df_sentiment.head(10)

## loc

In [2]:
table = pd.read_csv('Tweets_sentiment_16M.csv')
table.head(10)

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1,num_hashtags,sentiment_score
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - awww, t...",awww thats a bummer you shoulda got david carr...,thats a bummer you got carr of third day to do...,[],"['thats', 'a', 'bummer', 'you', 'got', 'carr',...","['thats', 'bummer', 'got', 'carr', 'third', 'd...",6,0,3
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his facebook by ...,is upset that he cant update his facebook by t...,is upset that he cant update his by it and mig...,[],"['is', 'upset', 'that', 'he', 'cant', 'update'...","['upset', 'cant', 'update', 'might', 'cry', 'r...",10,0,1
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@kenichan i dived many times for the ball. man...,i dived many times for the ball managed to sav...,i many times for the ball to save the rest go ...,[],"['i', 'many', 'times', 'for', 'the', 'ball', '...","['many', 'times', 'ball', 'save', 'rest', 'go']",6,0,2
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body itchy and like its on fire,[],"['my', 'whole', 'body', 'itchy', 'and', 'like'...","['whole', 'body', 'itchy', 'like', 'fire']",5,0,2
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...,no its not at all mad why am i here because i ...,[],"['no', 'its', 'not', 'at', 'all', 'mad', 'why'...","['mad', 'cant', 'see']",3,0,1
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@kwesidei not the whole crew,not the whole crew,not the whole crew,[],"['not', 'the', 'whole', 'crew']","['whole', 'crew']",2,0,2
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,need a hug,need a hug,need a hug,[],"['need', 'a', 'hug']","['need', 'hug']",2,0,5
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@loltrish hey long time no see! yes.. rains a...,hey long time no see yes rains a bit only a bi...,hey long time no see yes a bit only a bit fine...,[],"['hey', 'long', 'time', 'no', 'see', 'yes', 'a...","['hey', 'long', 'time', 'see', 'yes', 'bit', '...",9,0,4
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@tatiana_k nope they didn't have it,nope they didnt have it,nope they didnt have it,[],"['nope', 'they', 'didnt', 'have', 'it']","['nope', 'didnt']",2,0,1
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?,que me muera,me,[],['me'],[],0,0,3


In [3]:
# show where target 0 and sentiment score 5 or 4
table[(table['target'] == 0) & ((table['sentiment_score'] == 5) | (table['sentiment_score'] == 4))].head(10)

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1,num_hashtags,sentiment_score
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,need a hug,need a hug,need a hug,[],"['need', 'a', 'hug']","['need', 'hug']",2,0,5
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@loltrish hey long time no see! yes.. rains a...,hey long time no see yes rains a bit only a bi...,hey long time no see yes a bit only a bit fine...,[],"['hey', 'long', 'time', 'no', 'see', 'yes', 'a...","['hey', 'long', 'time', 'see', 'yes', 'bit', '...",9,0,4
10,0,1467812416,Mon Apr 06 22:20:16 PDT 2009,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing,spring break in plain city its snowing,spring break in plain city its,[],"['spring', 'break', 'in', 'plain', 'city', 'its']","['spring', 'break', 'plain', 'city']",4,0,4
11,0,1467812579,Mon Apr 06 22:20:17 PDT 2009,NO_QUERY,pardonlauren,i just re-pierced my ears,i just repierced my ears,i just my,[],"['i', 'just', 'my']",[],0,0,5
15,0,1467812799,Mon Apr 06 22:20:20 PDT 2009,NO_QUERY,HairByJess,@iamjazzyfizzle i wish i got to watch it with ...,i wish i got to watch it with you i miss you a...,i wish i got to watch it with you i miss you a...,[],"['i', 'wish', 'i', 'got', 'to', 'watch', 'it',...","['wish', 'got', 'watch', 'miss', 'premiere']",5,0,5
18,0,1467813579,Mon Apr 06 22:20:31 PDT 2009,NO_QUERY,starkissed,@lettya ahh ive always wanted to see rent lov...,ahh ive always wanted to see rent love the sou...,always to see rent love the,[],"['always', 'to', 'see', 'rent', 'love', 'the']","['always', 'see', 'rent', 'love']",4,0,5
19,0,1467813782,Mon Apr 06 22:20:34 PDT 2009,NO_QUERY,gi_gi_bee,@fakerpattypattz oh dear. were you drinking ou...,oh dear were you drinking out of the forgotten...,oh dear were you drinking out of the forgotten...,[],"['oh', 'dear', 'were', 'you', 'drinking', 'out...","['oh', 'dear', 'drinking', 'forgotten', 'table']",5,0,5
21,0,1467813992,Mon Apr 06 22:20:38 PDT 2009,NO_QUERY,swinspeedx,"one of my friend called me, and asked to meet ...",one of my friend called me and asked to meet w...,one of my friend me and to meet with her at mi...,[],"['one', 'of', 'my', 'friend', 'me', 'and', 'to...","['one', 'friend', 'meet', 'mid', 'valley', 'ti...",7,0,5
24,0,1467814192,Mon Apr 06 22:20:41 PDT 2009,NO_QUERY,Ljelli3166,blagh class at 8 tomorrow,blagh class at 8 tomorrow,class at tomorrow,[],"['class', 'at', 'tomorrow']","['class', 'tomorrow']",2,0,5
25,0,1467814438,Mon Apr 06 22:20:44 PDT 2009,NO_QUERY,ChicagoCubbie,i hate when i have to call and wake people up,i hate when i have to call and wake people up,i hate when i have to call and wake people up,[],"['i', 'hate', 'when', 'i', 'have', 'to', 'call...","['hate', 'call', 'wake', 'people']",4,0,5


In [21]:
# loc data
table.loc[0:20, ['text3', 'target', 'sentiment_score']]

,text3,target,sentiment_score
0,thats a bummer you got carr of third day to do...,0,3
1,is upset that he cant update his by it and mig...,0,1
2,i many times for the ball to save the rest go ...,0,2
3,my whole body itchy and like its on fire,0,2
4,no its not at all mad why am i here because i ...,0,1
5,not the whole crew,0,2
6,need a hug,0,5
7,hey long time no see yes a bit only a bit fine...,0,4
8,nope they didnt have it,0,1
9,me,0,3


In [24]:
table.loc[19, 'text3']

'oh dear were you drinking out of the forgotten table'

In [26]:
def sentiment_score_for3(text):
    if isinstance(text, str):
        # If input is a single string, tokenize it and encode it as a tensor
        tokens = tokenizer.encode(text, return_tensors='pt')
    elif isinstance(text, list):
        # If input is a list of strings, tokenize each string and concatenate the tensors
        tokens = torch.cat([tokenizer.encode(t, return_tensors='pt') for t in text])
    else:
        return -1
        
    result = model(tokens)
    # return int(torch.argmax(result.logits))+1
    # return mean of all the scores
    if int(torch.argmax(result.logits))+1 == 3:
        # choose the second largest score
        return int(torch.topk(result.logits, 2)[1][0][1])+1

In [30]:
# if where sentiment score = 3 find sentiment again 
table.loc[table['sentiment_score'] == 3, 'sentiment_score'] = table.loc[table['sentiment_score'] == 3, 'text3'].apply(lambda x: sentiment_score_for3(x))

In [31]:
table

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1,num_hashtags,sentiment_score
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - awww, t...",awww thats a bummer you shoulda got david carr...,thats a bummer you got carr of third day to do...,[],"['thats', 'a', 'bummer', 'you', 'got', 'carr',...","['thats', 'bummer', 'got', 'carr', 'third', 'd...",6,0,2
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his facebook by ...,is upset that he cant update his facebook by t...,is upset that he cant update his by it and mig...,[],"['is', 'upset', 'that', 'he', 'cant', 'update'...","['upset', 'cant', 'update', 'might', 'cry', 'r...",10,0,1
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@kenichan i dived many times for the ball. man...,i dived many times for the ball managed to sav...,i many times for the ball to save the rest go ...,[],"['i', 'many', 'times', 'for', 'the', 'ball', '...","['many', 'times', 'ball', 'save', 'rest', 'go']",6,0,2
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body itchy and like its on fire,[],"['my', 'whole', 'body', 'itchy', 'and', 'like'...","['whole', 'body', 'itchy', 'like', 'fire']",5,0,2
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...,no its not at all mad why am i here because i ...,[],"['no', 'its', 'not', 'at', 'all', 'mad', 'why'...","['mad', 'cant', 'see']",3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583221,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,just woke up. having no school is the best fee...,just woke up having no school is the best feel...,just woke up no school is the best feeling ever,[],"['just', 'woke', 'up', 'no', 'school', 'is', '...","['woke', 'school', 'best', 'feeling', 'ever']",5,0,5
1583222,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,thewdb.com - very cool to hear old walt interv...,thewdbcom very cool to hear old walt interviews,very cool to hear old walt,[],"['very', 'cool', 'to', 'hear', 'old', 'walt']","['cool', 'hear', 'old', 'walt']",4,0,5
1583223,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,are you ready for your mojo makeover? ask me f...,are you ready for your mojo makeover ask me fo...,are you ready for your mojo ask me for,[],"['are', 'you', 'ready', 'for', 'your', 'mojo',...","['ready', 'mojo', 'ask']",3,0,5
1583224,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,happy 38th birthday to my boo of alll time!!! ...,happy 38th birthday to my boo of alll time tup...,happy birthday to my boo of time,[],"['happy', 'birthday', 'to', 'my', 'boo', 'of',...","['happy', 'birthday', 'boo', 'time']",4,0,5


In [32]:
# show the number of each sentiment score
table['sentiment_score'].value_counts()

5    570833
1    446829
2    295957
4    269607
Name: sentiment_score, dtype: int64

In [33]:
# save the dataframe to csv file
table.to_csv('Tweets_sentiment_no3.csv', index=False)

In [34]:
# let 4-5 be positive, 1-2 be negative
table.loc[table['sentiment_score'] == 1, 'sentiment_score'] = 0
table.loc[table['sentiment_score'] == 2, 'sentiment_score'] = 0
table.loc[table['sentiment_score'] == 4, 'sentiment_score'] = 4
table.loc[table['sentiment_score'] == 5, 'sentiment_score'] = 4

In [35]:
table

,target,id,date,flag,user,text,text2,text3,hashtags,tokens,tokens_remove_stopwords,length_1,num_hashtags,sentiment_score
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - awww, t...",awww thats a bummer you shoulda got david carr...,thats a bummer you got carr of third day to do...,[],"['thats', 'a', 'bummer', 'you', 'got', 'carr',...","['thats', 'bummer', 'got', 'carr', 'third', 'd...",6,0,0
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his facebook by ...,is upset that he cant update his facebook by t...,is upset that he cant update his by it and mig...,[],"['is', 'upset', 'that', 'he', 'cant', 'update'...","['upset', 'cant', 'update', 'might', 'cry', 'r...",10,0,0
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@kenichan i dived many times for the ball. man...,i dived many times for the ball managed to sav...,i many times for the ball to save the rest go ...,[],"['i', 'many', 'times', 'for', 'the', 'ball', '...","['many', 'times', 'ball', 'save', 'rest', 'go']",6,0,0
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,my whole body itchy and like its on fire,[],"['my', 'whole', 'body', 'itchy', 'and', 'like'...","['whole', 'body', 'itchy', 'like', 'fire']",5,0,0
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",no its not behaving at all im mad why am i her...,no its not at all mad why am i here because i ...,[],"['no', 'its', 'not', 'at', 'all', 'mad', 'why'...","['mad', 'cant', 'see']",3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1583221,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,just woke up. having no school is the best fee...,just woke up having no school is the best feel...,just woke up no school is the best feeling ever,[],"['just', 'woke', 'up', 'no', 'school', 'is', '...","['woke', 'school', 'best', 'feeling', 'ever']",5,0,4
1583222,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,thewdb.com - very cool to hear old walt interv...,thewdbcom very cool to hear old walt interviews,very cool to hear old walt,[],"['very', 'cool', 'to', 'hear', 'old', 'walt']","['cool', 'hear', 'old', 'walt']",4,0,4
1583223,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,are you ready for your mojo makeover? ask me f...,are you ready for your mojo makeover ask me fo...,are you ready for your mojo ask me for,[],"['are', 'you', 'ready', 'for', 'your', 'mojo',...","['ready', 'mojo', 'ask']",3,0,4
1583224,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,happy 38th birthday to my boo of alll time!!! ...,happy 38th birthday to my boo of alll time tup...,happy birthday to my boo of time,[],"['happy', 'birthday', 'to', 'my', 'boo', 'of',...","['happy', 'birthday', 'boo', 'time']",4,0,4


In [36]:
# save the dataframe to csv file
table.to_csv('Tweets_sentiment_04.csv', index=False)

In [37]:
# show accurency 
from sklearn.metrics import accuracy_score
accuracy_score(table['target'], table['sentiment_score'])

0.6840255276252412